# support vector machine

In [30]:
# Binary classifier
# find the best hyperplane that separates classes

In [45]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, cross_validation, neighbors, svm
col_list = ['id', 'clump_thickness', 'unif_cell_size', 'unif_cell_shape', 'marg_adhesion', 'single_epith_cell_size', 'bare_nuclei', 'bland_chrom', 'norm_nucl', 'mitosis', 'Class' ]
df = pd.read_csv(r'C:\Users\Mfornaroli\Downloads\breast-cancer-wisconsin.data')
df.columns=col_list
df.replace('?', -99999, inplace=True) 
df.drop('id', axis=1, inplace=True)
X = np.array(df.drop(['Class'], 1))
y = np.array(df.Class)
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size = 0.2)
clf = svm.SVC()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy)

(698, 9)
(698,)
0.9714285714285714


vectors: direction and module. 
dot product (component by component, and sum)
creating decision boundaries
boundary: described by a vector? (perpendicular to it i guess?)
unknown point: described by another vector
Eq describing decision boundary: {dot product between vectors + bias = 0}
for one class the equation comes +1, for the other -1 (not clear)
need to find: decision function. now it gets...weird
we want to locate support vectors, apparently. equations to derive them. 
support vectors (+ and -1) --> make the equation be... 
x_+sv dot w + b = +1   for class +
x_-sv dot w + b = -1   for class -
width of the hyperplanes that 'go through' these support vectors? (distance between them...)*
support vectors: points that the two hyperplanes with the most distance
between them pass through
support vector definition: feature set that determine the hyperplane... 
if we move it, it affects best separation. 
width/2 will give us our plane!*
we want to maximize that width, to get the best separating hyperplane. 
width = to be maximized = (x_+sv - x_-sv)*(w/module(w)) 
= (...) = 2/module(w)


so: our classification will be done:
(weight vector) dot product (feature vector)
best 'line' defined by weight vector w. 
distance from separating line: how confident we are in our prediction. 
we want to  have the largest 'margin'.
ok so i ve verified: we can define a plane as:
(vector X dot normal_vect + bias = 0 )
given a point A (vecotor), algebra --> distance between point A and plane turn out to be:
norm_vect dot vect_A + bias (same bias!)
check: https://www.youtube.com/watch?v=ax8LxRZCORU
and https://www.youtube.com/watch?v=YuyeOErjrOM
and also: https://www.youtube.com/watch?v=_PwhiWxHK8o&t=2s
so now that we know how to express the distance, lets find the best plane, with the highest margin (distance...)
prediction = sign(norm_vect dot vect_A + bias)
confidence = norm of distance
we want to find w (and b?) so that the smalles margin (distance module) is as large as possible (kind of)
support vectors: dots that are determined to be nearest to the best dec boundary (given a definition of optimization that includes the possibility of outliers, that are allowed within a certain budget)
so: best line can be found... looking somehow at the support vectors only. 
JUST CHECK THE INTERNET FOR EXHAUSTIVE EXPLANATIONS

In [46]:
# solving the optimization problem
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
style.use('ggplot')

In [48]:
data_dict = {-1: np.array([[1,7],
                          [2,8],
                          [3,8]]), 
             +1: np.array([[5,1],
                          [6,-1],
                          [7,3]])}
print(data_dict)

{-1: array([[1, 7],
       [2, 8],
       [3, 8]]), 1: array([[ 5,  1],
       [ 6, -1],
       [ 7,  3]])}


In [6]:
# building our SVM class
class Supp_Vect_Machine:
    def __init__(self, visualization=True):
        self.visualization = visualization
        self.colors = {1: 'r', -1: 'b'}
        if self.visualization:
            self.fig = plt.figure()
            self.ax = self.fig.add_subplot(1,1,1) # check out matplotlib tutorial man
   
    #training step, dude. optimization: there are much better ways  
    def fit(self, data):
        self.data = data # now the algo can reference data
        opt_dict = {}
        transforms = [[1,1],  # basically, each time we have a vector, we want to transform it       
                     [-1,1],  
                     [-1,-1],
                     [1,-1]]
        all_data = []
        for yi in self.data:
            for featureset in self.data[yi]:
                for feature in featureset:
                    all_data.append(feature)
        # searching for max and min values
        self.max_feature_value = max(all_data)
        self.min_feature_value = min(all_data)
        all_data = None
        # first big steps, then smaller as we approach the minimum
        step_sizes = [self.max_feature_value * 0.1,   # big steps
                     self.max_feature_value * 0.01,
                      # point of expense:
                     self.max_feature_value * 0.001]
        # extremely expensive
        b_range_multiple = 5   # b doesnt need to be as precise as w
        b_multiple = 5
        latest_optimum = self.max_feature_value * 10  # first elemnt 
        
        # begin actual stepping process
        # we dont need to take as small steps with b as we do with w
        for step in spet_sizes:
            w = np.array([latest_optimum, latest_optimum])
            # we can do it because its convex, we know its gonna have a solution.. 
            optimized = False # will stay false until we have no more steps down to take
            while not optimized:
                # we want to have minimum module of w and maximum bias b
                for b in np.arange(-1*(self.max_feature_value*b_range_multiple), 
                                   self.max_feature_value*b_range_multiple,
                                  step*b_multiple):
                    for transformation in transforms:
                        # we are going to run through a bunch of w that have components between -80 and +80 in our case
                        w_t = w*transformation
                        found_option = True
                        # weakest link in SVM basically
                        # SMO attempts to fix it.... 
                        # we know have to run the calculation for all of the data!
                        # yi(w . xi + b) >= 1   this is our constraint
                        for i in self.data:  #i is the class
                            for xi in self.data[i]:  
                                yi = i
                                if not yi*(np.dot(w_t, xi)+b) >= 1:
                                    found_option = False
                                    # and maybe put a break here?
                                    
                        if found_option: # everything checked out
                            opt_dict[np.linalg.norm(w_t)] = [w_t, b]
                    if w[0] < 0:
                        optimized = True
                        print("optimized a step.")
                    else:
                        w = w - step
                norms = sorted([n for n in opt_dict])
                
                opt_choice = opt_dict[norms[0]]
                self.w = opt_choice[0]
                self.b = opt_choice[1]
                latest_optimum = opt_choice[0][0] + step*2
    
    def predict(self, feature):
        # sign(x dot w + b)
        classification = np.sign(np.dot(np.array(features), self.w) + self.b)
        if classification != 0 and self.visualization:
            self.ax.scatter(features[0], features[1], marker='*', c=self.colors[classification])
        return classification
    
    def visualize(self):
        [[self.ax.scatter(x[0], x[1], s=100, colors=self.colors[i]) for x in data_dict[i]] for i in data_dict]
        
    def hyperplane(x, w, b, v):
        

In [7]:
# it's really getting too long and complicated... to be completed maybe another day?